In [58]:
from fastai_old.text import *
import html
import spacy 

spacy.load('en_core_web_sm')


LM_PATH = Path('data_inf/custom_lm/')
# LM_PATH = Path('data_inf/custom_lm/test_vocab/')
# LM_PATH = Path('data_inf/pretrained/')
LM_PATH.mkdir(exist_ok=True)

em_sz,nh,nl = 400,1150,3

wd=1e-7
bptt=70
bs=1
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

# itos_name = 'itos_wt103.pkl'
# model_name = 'pretrained'

# itos_name = 'itos_imdb.pkl'
# model_name = 'lm_last_ft'

# itos_name = 'itos_alice_nop.pkl'
# model_name = 'alice_nop_lm_30epochs'

# itos_name = 'itos_alice_processed.pkl'
# model_name = 'alice_processed_lm_30epochs'

itos_name = 'itos_first_three_decades_test.pkl'
model_name = 'first_three_decades_test_lm_30epochs'

itos2 = pickle.load((LM_PATH/itos_name).open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

trn_lm = np.array([list([0,0])])
val_lm = np.array([list([0,0])])

vs=len(itos2)

trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(LM_PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.load(model_name)
m = learner.model
vs


def generate_text(m, s, l=20):
    m[0].bs=1  # Set batch size to 1
    m.eval()  # Turn off dropout
    m.reset()  # Reset hidden state
    m[0].bs=bs  # Put the batch size back to what it was

    ss = s.lower().split()
    si = [stoi2[w] for w in ss]
    g = np.array([si])
    f = torch.LongTensor(g)
    t = torch.autograd.Variable(f)
    
    res,*_ = m(t)

    print(s, end=' ')
    for i in range(l):
#         n = res[-1].topk(5)[1]  # top word
        n = torch.multinomial(res[-1].exp(), 2)  # drawing from probability distribution
        n = n[1] if n.data[0]==0 else n[0]
        print(itos2[int(n)], end=' ')
        res,*_ = m(n.unsqueeze(0).unsqueeze(0))  # sometimes need an extra .unsqueeze(0)
    print('...')
    
    
generate_text(m, "britain", l=250)

britain and america were making a great deal to understand each other . this system , this one of english genius , has been inspired by a great deal about our friendship and friendship , and in spite of all the great qualities of the old and the splendid , the way people feel about each other 's welcome , the way people feel about each other 's happiness , and of their loyalty and affection each other makes a difference . in spite of all the scientific and continuing needs of the world and for some of the most practical and happier effort that has come , the genius of scientists and engineers must be immense . you have shown by the way in which you have celebrated the jubilee that this was not an experience of happiness . my father , king george the valiant , developed these beliefs and ideas , but we do not always reflect that these things are not always easy . for my husband and myself and for our children , the year that is passing has celebrated the centenary of the festival of pea